<a href="https://colab.research.google.com/github/NavaneethKrishnanB/FacExp/blob/master/facExp_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Mar  9 13:02:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 42kB/s 


In [ ]:
from tensorflow.keras.layers import Lambda,Dense,Flatten,Input
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE=[224,224]
train_path="/content/drive/MyDrive/images/train"
test_path="/content/drive/MyDrive/images/validation"
vgg16 = VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

58892288/58889256 [==============================] - 2s 0us/step


In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
folders=glob(train_path+"/*")
print(len(folders))

5


In [ ]:
x = Flatten()(vgg16.output)
prediction = Dense(len(folders),activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg16.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer='adam',
             metrics=['accuracy'],
             loss='categorical_crossentropy')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  rotation_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 25190 images belonging to 5 classes.


In [ ]:
test_set=test_datagen.flow_from_directory(
                                          test_path,
                                          target_size=(224,224),
                                          batch_size=32,
                                          class_mode='categorical',
                                          shuffle=True
)

Found 6158 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
earlystop=EarlyStopping(monitor='val_loss',
                        patience=5,
                        verbose=1,
                        restore_best_weights=True)

In [ ]:
checkpoint=ModelCheckpoint('face_exp_vgg16.h5',
                          monitor='val_loss',
                          mode='min',
                          save_best_only=True,
                          verbose=1)

In [ ]:
history = model.fit(x=train_set,
                   validation_data=train_set,
                   epochs=30,
                   callbacks=[checkpoint,earlystop],
                   steps_per_epoch=len(train_set),
                   validation_steps=len(test_set))

Epoch 1/30
788/788 [==============================] - 15402s 19s/step - loss: 1.6153 - accuracy: 0.3625 - val_loss: 1.3330 - val_accuracy: 0.4626

Epoch 00001: val_loss improved from inf to 1.33303, saving model to face_exp_vgg16.h5
Epoch 2/30
788/788 [==============================] - 401s 509ms/step - loss: 1.4153 - accuracy: 0.4459 - val_loss: 1.3099 - val_accuracy: 0.4833

Epoch 00002: val_loss improved from 1.33303 to 1.30992, saving model to face_exp_vgg16.h5
Epoch 3/30
788/788 [==============================] - 386s 490ms/step - loss: 1.3929 - accuracy: 0.4649 - val_loss: 1.3392 - val_accuracy: 0.4780

Epoch 00003: val_loss did not improve from 1.30992
Epoch 4/30
788/788 [==============================] - 384s 488ms/step - loss: 1.3325 - accuracy: 0.4911 - val_loss: 1.4269 - val_accuracy: 0.4576

Epoch 00004: val_loss did not improve from 1.30992
Epoch 5/30
788/788 [==============================] - 382s 484ms/step - loss: 1.3309 - accuracy: 0.4893 - val_loss: 1.3260 - val_accur